# Managing the Wurtz Benches

In [1]:
import gymnasium as gym
import chemistrylab
import matplotlib,time
import numpy as np
from matplotlib import pyplot as plt
from chemistrylab.util import Visualization
from IPython.display import display,clear_output
from chemistrylab.util.ActionDoc import generate_manager_table
from chemistrylab.lab.manager import CONFIG_PATH
Visualization.use_mpl_dark(size=2)

# Manager Actions

The table below outline what actions are available to the agent in the Manager-v0 environment.
Let's break down the table:

- **Action Index**: This is a unique identifier for each action. Here multiple actions may have the same index because actions are context-dependent.

- **Required Bench**: This column indicates the bench that is required for each action. The term "None" means the action doesn't require a specific bench setup, while terms like "Reaction," "Distillation," and "Extraction" indicate actions are only available in these benches.

- **Function Call**: This column describes the function call associated with each action. These functions can also be called directly when not using the Manager in a reinforcement learning setup.

In [2]:
print(generate_manager_table(CONFIG_PATH+"/wurtz.json"))

+----------------+------------------+----------------------------------------------+
|   Action Index | Required Bench   | Function Call                                |
+================+==================+==============================================+
|              0 | None             | set_cur_bench(idx = 0)                       |
+----------------+------------------+----------------------------------------------+
|              1 | None             | set_cur_bench(idx = 1)                       |
+----------------+------------------+----------------------------------------------+
|              2 | None             | set_cur_bench(idx = 2)                       |
+----------------+------------------+----------------------------------------------+
|              3 | None             | end_experiment()                             |
+----------------+------------------+----------------------------------------------+
|              4 | None             | swap_vessels(bench_idx = -1

In [3]:
manager = gym.make('Manager-v0')
_ = manager.reset()

C:\Users\sprag\anaconda3\envs\rllib\lib\site-packages\gymnasium\utils\passive_env_checker.py:164: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
C:\Users\sprag\anaconda3\envs\rllib\lib\site-packages\gymnasium\utils\passive_env_checker.py:188: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


# Setting the Bench

In [4]:
print("Current Bench:",manager.current_bench)

print("Setting Bench to 1 (Distillation)")
manager.set_cur_bench(1)
print("Current Bench:",manager.current_bench)

print("Setting Bench to 0 (Reaction)")
manager.step(0)
print("Current Bench:",manager.current_bench)

print("Setting Bench to 2 (Extraction)")
manager.step(2)
print("Current Bench:",manager.current_bench)

print("Setting Bench to 0 (Reaction)")
manager.set_cur_bench(0)
print("Current Bench:",manager.current_bench)

Current Bench: None
Setting Bench to 1 (Distillation)
Current Bench: 1
Setting Bench to 0 (Reaction)
Current Bench: 0
Setting Bench to 2 (Extraction)
Current Bench: 2
Setting Bench to 0 (Reaction)
Current Bench: 0


C:\Users\sprag\anaconda3\envs\rllib\lib\site-packages\gymnasium\utils\passive_env_checker.py:164: UserWarning: WARN: The obs returned by the `step()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
C:\Users\sprag\anaconda3\envs\rllib\lib\site-packages\gymnasium\utils\passive_env_checker.py:188: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


# Moving vessels between benches

Moving vessels around involves calling swap_vessel(), insert_vessel(), create_vessel(), and dispose_vessel()

In [5]:
#resetting the manager and setting the current bench to the reaction bench
_ = manager.reset()
manager.step(0)


print("Initial Reaction",manager.benches[0].shelf,"\n")

print("Moving the reaction vessel into the manager's hand")
#Moving vessels using the swap_vessels function
manager.swap_vessels(bench_idx=0, vessel_idx=0)

print("Reaction",manager.benches[0].shelf)
print("Hand:", manager.hand[0],"\n")

print("Placing the reaction vessel into the manager's shelf")
#swap_vessels with bench_idx=-1 uses the manager's personal shelf instead of a benches shelf
manager.swap_vessels(bench_idx=-1, vessel_idx=0)

print("Manager",manager.shelf)
print("Hand:", manager.hand,"\n")

Initial Reaction Shelf: (Reaction Vessel, 1-chlorohexane Vessel, 2-chlorohexane Vessel, 3-chlorohexane Vessel, Na Vessel) 

Moving the reaction vessel into the manager's hand
Reaction Shelf: (1-chlorohexane Vessel, 2-chlorohexane Vessel, 3-chlorohexane Vessel, Na Vessel)
Hand: Reaction Vessel 

Placing the reaction vessel into the manager's shelf
Manager Shelf: (Reaction Vessel)
Hand: Empty Shelf 



In [6]:
print("Moving the 1-chlorohexane vessel into the manager's hand")
# Action 9 is equivalent to swap_vessels(bench_idx=0, vessel_idx=0)
manager.step(9)

print("Reaction",manager.benches[0].shelf)
print("Hand:", manager.hand[0],"\n")


print("Placing the 1-chlorohexane vessel into the manager's shelf")
# Action 5 is equivalent to swap_vessels(bench_idx = -1, vessel_idx = 1)
manager.step(5)

print("Manager",manager.shelf)
print("Hand:", manager.hand,"\n")

Moving the 1-chlorohexane vessel into the manager's hand
Reaction Shelf: (2-chlorohexane Vessel, 3-chlorohexane Vessel, Na Vessel)
Hand: 1-chlorohexane Vessel 

Placing the 1-chlorohexane vessel into the manager's shelf
Manager Shelf: (Reaction Vessel, 1-chlorohexane Vessel)
Hand: Empty Shelf 



# Running a Bench

You can run a bench with a predefined policy via the run_bench() function. 


Note: If the bench doesn't have a valid vessel setup then nothing happens

In [7]:
#resetting the manager and setting the current bench to the reaction bench
_ = manager.reset()
manager.set_target("CCCCCCCCCCCC")
_ = manager.step(0)

print("Initial Reaction Vessel:")
display(manager.benches[0].shelf[0].get_material_dataframe())

print("Running the reaction bench. . .")
manager.run_bench(bench_idx = 0, policy_idx = 0)

print("Final Reaction Vessel:")
display(manager.benches[0].shelf[0].get_material_dataframe())


manager.step(9)
# step(1) sets the current bench to distillation
manager.step(1)
manager.step(9)
manager.step(4)


print("Running the distillation bench. . .")
# This runs the distillation bench (since it's the current bench selected)
manager.step(8)

print("Distillation Bench output:")

manager.step(9)
manager.step(9)

display(manager.hand[0].get_material_dataframe())

Initial Reaction Vessel:


,Smiles,Amount,Phase,Solute,Solvent
diethyl ether,CCOCC,4,l,False,False


Running the reaction bench. . .
Final Reaction Vessel:


,Smiles,Amount,Phase,Solute,Solvent
diethyl ether,CCOCC,4.000000,l,False,False
1-chlorohexane,CCCCCCCl,0.101284,l,False,False
Na,[Na+],2.101284,s,False,False
dodecane,CCCCCCCCCCCC,0.449358,l,False,False
NaCl,[Na+].[Cl-],0.898716,s,False,False


Running the distillation bench. . .
Distillation Bench output:


,Smiles,Amount,Phase,Solute,Solvent
diethyl ether,CCOCC,0.000000,l,False,True
1-chlorohexane,CCCCCCCl,0.101284,l,False,False
dodecane,CCCCCCCCCCCC,0.449358,l,True,False


 # Ending the Experiment
 
 At the end of the experiment (either due to taking the 'end experiment' action or timing out), a reward is given based off of the amount and purity of the target material contained in the manager's shelf.

In [8]:
# Move our output vessel into the manager's shelf
manager.step(4)

obs, rew, done, info, _ = manager.step(3)

print(rew, done)

0.366703527308431 True
